In [2]:
from sympy import mod_inverse

In [33]:
p = 5
a = 2
b = 3

In [4]:
class ECPoint:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __eq__(self, other):
        return isinstance(other, ECPoint) and self.x == other.x and self.y == other.y

    def __neg__(self):
        return ECPoint(self.x, (-self.y) % p)

    def is_infinity(self):
        return self.x is None and self.y is None

    def __repr__ (self):
        if self.is_infinity():
            return "O"
        return f"({self.x}, {self.y})"

In [5]:
O = ECPoint(None, None)

In [6]:
def ec_add(P, Q):
    if P.is_infinity():
        return Q
    if Q.is_infinity():
        return P
    if Q == -P:
        return O
    if P != Q:
        try:
            l = ((Q.y - P.y) * mod_inverse(Q.x - P.x, p)) % p
        except ValueError:
            return O
    else:
        if P.y == 0:
            return O
        l = ((3 * P.x**2 + a) * mod_inverse(2 * P.y, p)) % p

    x_r = (l**2 - P.x - Q.x) % p
    y_r = (l * (P.x - x_r) - P.y) % p
    return ECPoint(x_r, y_r)


In [7]:
def ec_mul(P, n):
    R = O
    for i in range(n):
        R = ec_add(R, P)
    return R

In [8]:
def list_points():
    points = [O]
    for x in range(p):
        rhs = (x**3 + a*x + b) % p
        for y in range(p):
            if (y*y) % p == rhs:
                points.append(ECPoint(x, y))
    return points

In [34]:
list_points()

[O, (1, 1), (1, 4), (2, 0), (3, 1), (3, 4), (4, 0)]

In [38]:
G = ECPoint(1, 1)
for i in range(7):
    print(ec_mul(G, i))

O
(1, 1)
(3, 4)
(2, 0)
(3, 1)
(1, 4)
O


In [39]:
k = 3
P = ec_mul(G, 3)

In [40]:

mat = list(list(range(7)) for _ in range(7))
for i in range(7):
    for j in range(7):
        mat[i][j] = ec_add(ec_mul(P, i), ec_mul(G, j))
mat

[[O, (1, 1), (3, 4), (2, 0), (3, 1), (1, 4), O],
 [(2, 0), (3, 1), (1, 4), O, (1, 1), (3, 4), (2, 0)],
 [O, (1, 1), (3, 4), (2, 0), (3, 1), (1, 4), O],
 [(2, 0), (3, 1), (1, 4), O, (1, 1), (3, 4), (2, 0)],
 [O, (1, 1), (3, 4), (2, 0), (3, 1), (1, 4), O],
 [(2, 0), (3, 1), (1, 4), O, (1, 1), (3, 4), (2, 0)],
 [O, (1, 1), (3, 4), (2, 0), (3, 1), (1, 4), O]]

In [41]:
# 行列からすべての(3,1)のインデックスを取得
target_point = ECPoint(3, 1)
indices = []

for i in range(7):
    for j in range(7):
        if mat[i][j] == target_point:
            indices.append((i, j))

print(f"(3,1)が見つかったインデックス: {indices}")
print(f"合計{len(indices)}個見つかりました")

(3,1)が見つかったインデックス: [(0, 4), (1, 1), (2, 4), (3, 1), (4, 4), (5, 1), (6, 4)]
合計7個見つかりました
